# Festival Playlists

In [1]:
import os
import numpy as np
import pandas as pd
import requests
import json
import spotipy
from IPython.display import display

1. Use the Songkick API to get all the bands playing the festival
2. Use the Setlist.FM API to get the setlists
3. Use the Spotify API to create the playlists and add all the songs

### Set API credentials

In [2]:
setlistfm_api_key = os.getenv('SETLISTFM_API_KEY')
spotify_client_id = os.getenv('SPOTIFY_CLIENT_ID')
spotify_client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

### Setlist FM

#### Plan of action
1. Given a lineup (list of band names), get their Musicbrainz identifiers (`mbid`) via `https://api.setlist.fm/rest/1.0/search/artists`
2. Retrieve the setlists for each artist using their `mbid` via `https://api.setlist.fm/rest/1.0/artist/{artist_mbid}/setlists
`

In [3]:
lineup = pd.read_csv(
    '/Users/adrialuz/Desktop/repos/festival-playlists/data/Glasto2023.csv', header=None, names=['band'], encoding="ISO-8859-1"
)['band'].values

In [4]:
len(lineup)

38

In [5]:
lineup

array(['The Hives', 'Yaya Bey', 'Los Bitchos', 'Unknown Mortal Orchestra',
       'Gabriels', 'The Comet Is Coming', 'Joey Bada$$', 'Young Fathers',
       'Fred Again', 'Arctic Monkeys', 'Hot Chip', 'Wunderhorse',
       'Sudan Archives', 'The Murder Capital', 'Jockstrap', 'Shame',
       'Ezra Collective', 'Flohio', 'Lizzo', 'Rina Sawayama',
       'Lana Del Rey', 'Loyle Carner', 'Guns N Roses',
       'Christine and the Queens', 'Japanese Breakfast', 'CMAT',
       'Black Country New Road', 'Cat Stevens', 'Blondie', 'Viagra Boys',
       'Lil Nas X', 'The War On Drugs', 'Caroline Polachek', 'Thundercat',
       'Elton John', 'Queens of the Stone Age', 'Phoenix', 'Alt-J'],
      dtype=object)

In [6]:
artists_url = 'https://api.setlist.fm/rest/1.0/search/artists'

In [7]:
lineup_mbids = []
not_found = []

for name in lineup:
    req = requests.get(artists_url,
                       headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                       params={'artistName': name, 'p': 1, 'sort': 'relevance'}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(artists_url,
                           headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                           params={'artistName': name, 'p': 1, 'sort': 'relevance'}
        )
        i += 1
    
    if req.ok:
        artist_response = req.json()['artist']
        num_artists = len(artist_response)
        if num_artists > 1:
            for i in range(num_artists):
                if artist_response[i]['name'].lower() == name.lower():
                    mbid = artist_response[i]['mbid']
                    lineup_mbids.append({'name': name, 'mbid': mbid})
                    break
        elif num_artists == 1:
            mbid = artist_response[0]['mbid']
            lineup_mbids.append({'name': name, 'mbid': mbid})
        elif num_artists == 0:
            print(f'No results I think for {name}')
        else:
            print(f'WTF {name}?')
                    
    else:
        print(f'Couldn\'t find {name}')
        not_found.append(name)

In [8]:
lineup_mbids

[{'name': 'The Hives', 'mbid': '487bfd74-71bf-46dd-b89c-80b7a0f06f2f'},
 {'name': 'Yaya Bey', 'mbid': 'f55859c9-0642-47dc-8c68-4aa2db484b23'},
 {'name': 'Los Bitchos', 'mbid': '685051dc-49ad-496a-acab-57ad1ad1e491'},
 {'name': 'Unknown Mortal Orchestra',
  'mbid': 'e2305342-0bde-4a2c-aed0-4b88694834de'},
 {'name': 'Gabriels', 'mbid': 'c6bbc197-ce7b-48c7-8690-71a149073429'},
 {'name': 'The Comet Is Coming',
  'mbid': '2b82372f-8c85-4b47-a9fc-36fe4d4df5b4'},
 {'name': 'Joey Bada$$', 'mbid': '9757ae7d-44b1-4d63-81ce-b463fe27b9ae'},
 {'name': 'Young Fathers', 'mbid': '24aa5c20-a275-468a-93a9-b3d96b344b6c'},
 {'name': 'Arctic Monkeys', 'mbid': 'ada7a83c-e3e1-40f1-93f9-3e73dbc9298a'},
 {'name': 'Hot Chip', 'mbid': 'd8915e13-d67a-4aa0-9c0b-1f126af951af'},
 {'name': 'Wunderhorse', 'mbid': '5126db33-86ad-469d-88fb-4158350e1515'},
 {'name': 'Sudan Archives', 'mbid': '6ce5eca0-4ad6-49e0-a5ab-e11482d85324'},
 {'name': 'The Murder Capital',
  'mbid': '41e30723-07ab-40fd-8f6b-983981a76d8b'},
 {'name

In [9]:
not_found

[]

In [10]:
artist_setlist = []

for a in lineup_mbids:
    songs_played = []
    mbid = a['mbid']
    setlists_url = f'https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists'
    
    req = requests.get(setlists_url,
             headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
             params={'p': 1}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(setlists_url,
                 headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                 params={'p': 1}
        )
        i += 1
            
    if req.ok:
    
        setlist_response = req.json()['setlist']
        num_setlists = len(setlist_response)

        for i in range(num_setlists):
            setlist = setlist_response[i]['sets']['set']
            num_sections = len(setlist)
            total_songs = []
            for p in range(num_sections):
                total_songs += setlist[p]['song']
            num_songs = len(total_songs)

            for i in range(num_songs):
                song = total_songs[i]
                song_title = song['name']
                # if the song is a cover add the original artist to the song title
                if 'cover' in song:
                    song_title += ' {}'.format(song['cover']['name'])
                songs_played.append(song_title)
                                           
        most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)

        artist_setlist.append({
            'artist': a['name'],
            'setlist': most_played_songs
        })
    else:
        not_found.append(a['name'])

/var/folders/5g/z2_s3_7d75qgyjp72n_8q0p40000gq/T/ipykernel_35717/1942038417.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)


In [11]:
not_found

[]

In [12]:
artist_setlist[:5]

[{'artist': 'The Hives',
  'setlist': ['Main Offender',
   'Good Samaritan',
   'Come On!',
   'Tick Tick Boom',
   'Walk Idiot Walk',
   'Hate to Say I Told You So',
   'Stick Up',
   'Go Right Ahead',
   "I'm Alive",
   'Bogus Operandi',
   'Countdown to Shutdown',
   'Rigor Mortis Radio',
   'Paint a Picture',
   "Won't Be Long",
   'Trapdoor Solution']},
 {'artist': 'Yaya Bey', 'setlist': []},
 {'artist': 'Los Bitchos',
  'setlist': ['Good to Go!',
   'Lindsay Goes to Mykonos',
   'The Link Is About to Die',
   'Tequila The Champs',
   'Las panteras',
   'Pista (Fresh Start)',
   'Try the Circle!',
   'HI!',
   'Change of Heart',
   '',
   'La Bomba',
   'Trapdoor King Gizzard & The Lizard Wizard',
   'Tripping at a Party',
   'Naughty Lil Irish Leaf',
   'Tropico']},
 {'artist': 'Unknown Mortal Orchestra',
  'setlist': ['The Garden',
   'Nadja',
   'Layla',
   'Multi-Love',
   'That Life',
   'Hunnybee',
   'Swim and Sleep (Like a Shark)',
   "Can't Keep Checking My Phone",
   'Me

In [13]:
setlist_lengths = []
short_or_empty_setlist = []

for i in range(len(artist_setlist)):
    n_songs = len(artist_setlist[i]['setlist'])
    setlist_lengths.append({
        'artist': artist_setlist[i]['artist'],
        'n_songs': n_songs
    })
    
    if n_songs < 5:
        short_or_empty_setlist.append(artist_setlist[i]['artist'])

In [14]:
len(short_or_empty_setlist)

1

In [15]:
short_or_empty_setlist

['Yaya Bey']

### Spotify

In [16]:
username = 'adrialuz'
scope = 'playlist-modify-public'

token = spotipy.util.prompt_for_user_token(username, scope, redirect_uri='http://localhost:9090')

sp = spotipy.Spotify(auth=token)

sp.trace = False

In [17]:
spotify_ids = []
for a in short_or_empty_setlist:
    search_result = sp.search(
        f'artist:{a}', limit=5, type='artist', market='GB'
    )['artists']['items']
    
    if search_result:
        for i in range(len(search_result)):
            name = search_result[i]['name']
            if name.lower() == a.lower():
                artist_id = search_result[i]['id']
                spotify_ids.append(artist_id)
                break
            else:
                pass
    else:
        print(f'Couldn\'t find {a} on Spotify.')

In [18]:
spotify_ids[:5]

['6tpaMMCs8X6o8j9H5OmWmT']

In [19]:
sp.artist('59xdAObFYuaKO2phzzz07H')['name']

'Special Request'

In [20]:
popular_songs = []

for artist_id in spotify_ids:
    search_results = sp.artist_top_tracks(artist_id, country='GB')['tracks']

    top_songs = []
    if search_results:
        for i in range(len(search_results)):
            song_name = search_results[i]['name']
            top_songs.append(song_name)
        popular_songs.append({
            'artist': sp.artist(artist_id)['name'],
            'setlist': top_songs
        })
    else:
        print(artist_id, sp.artist(artist_id)['name'])

In [21]:
popular_songs[:5]

[{'artist': 'Yaya Bey',
  'setlist': ['you up?',
   'meet me in brooklyn',
   'intro',
   'pour up',
   'september 13th',
   'on the pisces moon',
   'fxck it then',
   'mama loves her son',
   'ascendent (mother fxcker)',
   'alright']}]

Get the URI codes for each track

In [22]:
uris = []
missing_songs = []

for a in (artist_setlist + popular_songs):
    artist = a['artist']
    setlist = a['setlist']
    for s in setlist:
        s = s.replace(',', '').replace('\'', '').replace('"', '').replace('.', '').replace(
            '?', '').replace(')', '').replace('(', '').replace('/', '').replace(
            '\\', '').replace('&', '').replace('-', '')
        items = sp.search(q=f'artist:{artist} track:{s}', limit=1)['tracks']['items']
        if items:
            uri = items[0]['id']
            uris.append(uri)
        else:
            items = sp.search(q=f'track:{s}', limit=1)['tracks']['items']
            if items:
                if items != [None]:
                    uri = items[0]['id']
                    uris.append(uri)
            else:
                missing_songs.append({
                    'artist': artist,
                    'song': s
                })

In [24]:
len(uris)

506

In [25]:
len(missing_songs)

21

In [26]:
missing_songs

[{'artist': 'The Hives', 'song': 'Trapdoor Solution'},
 {'artist': 'Los Bitchos', 'song': 'Naughty Lil Irish Leaf'},
 {'artist': 'Unknown Mortal Orchestra', 'song': 'Necessary Evil  Monki'},
 {'artist': 'Gabriels', 'song': 'Professional  the way we were'},
 {'artist': 'The Comet Is Coming', 'song': 'Shabaka Hutchings Sax Solo'},
 {'artist': 'Hot Chip', 'song': 'FreakoutRelease'},
 {'artist': 'Wunderhorse', 'song': 'Oprah Winfrey Is This Love'},
 {'artist': 'Lana Del Rey',
  'song': 'Change  Black Beauty  Young and Beautiful  Ride'},
 {'artist': 'CMAT', 'song': 'Where Are Your Kids Tonight'},
 {'artist': 'Black Country New Road', 'song': 'Nancy Tries to Take the Night'},
 {'artist': 'Black Country New Road', 'song': 'TurbinesPigs'},
 {'artist': 'Black Country New Road', 'song': '247 365 British Summer Time'},
 {'artist': 'Cat Stevens', 'song': 'SunC79'},
 {'artist': 'Blondie', 'song': 'Hanging on the Telephone The Nerves'},
 {'artist': 'Lil Nas X', 'song': 'Down Souf Hoes'},
 {'artist':

In [27]:
divisor = int(np.floor(len(uris) / np.ceil(len(uris) / 100)))
times = int(np.floor(len(uris) / divisor))

In [28]:
for i in range(times):
    subset = uris[divisor*i:divisor*(i+1)]
    sp.user_playlist_add_tracks(username, playlist_id='7kiHwhbzlfB4Ut01CMfp4P',
                                tracks=subset)